In [ ]:
import sys
import os
# Add the parent directory to the path to import src
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from collections import deque
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from sklearn.metrics import mean_squared_error

# Import from local src package
from src.models import TransformerEncoder_version2, PositionalEncoding
from src.data import (TimeSeriesDataset, load_ohio_series_train, create_5fold_splits, create_5fold_splits_T1DEXI,
                     convert_to_datetime, load_train_data_by_fold, load_train_data_by_fold_T1DEXI,
                     split_into_continuous_series,
                     create_population_splits, create_train_val_datasets)
from src.train import train_model, evaluate_model
from src.utils import (save_model, load_model, load_model_population,
                      evaluate_and_save_metrics, evaluate_and_save_metrics_population,
                      evaluate_and_save_metrics_diatrend, evaluate_and_save_metrics_T1DEXI)

In [ ]:
model_dir = '../saved_models/'
evaluation_dir = '../evaluation/'

os.makedirs(model_dir, exist_ok=True)
os.makedirs(evaluation_dir, exist_ok=True)

# Train on DiaTrend

## 5 fold CV

In [ ]:
data_dir = '../../../ReproGenBG_Dataset/diatrend_processed/'

fold_splits = create_5fold_splits(data_dir)
print(fold_splits)

## model train

In [ ]:
fold_lst = fold_splits.keys()
print(fold_lst)

for fold in fold_lst:
    train_df = load_train_data_by_fold(fold, fold_splits, data_dir)
    print(fold, '\ntrain data shape:', train_df.shape)
    # break

    # Move model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Set hyperparameters
    past_sequence_length = 12
    future_offset = 6
    batch_size = 64
    max_interval_minutes = 30

    # Train model
    model = TransformerEncoder_version2(
      past_seq_len=past_sequence_length,
      num_layers=1,
      d_model=512,
      nhead=4,
      input_dim=1,
      dropout=0.2
    )
    model = model.to(device)

    # Create datasets
    train_series_list = []
    for uid in train_df['USUBJID'].unique():
        cur_df = train_df[train_df['USUBJID'] == uid]
        cur_df.drop(columns=['USUBJID'], inplace=True)
        series_list = split_into_continuous_series(cur_df, past_sequence_length, future_offset, max_interval_minutes)
        train_series_list.extend(series_list)

    train_dataset, val_dataset = create_train_val_datasets(
      train_series_list,
      train_ratio=0.8,
      past_seq_len=past_sequence_length,
      future_offset=future_offset
    )

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Train model
    train_losses, val_losses = train_model(
      model=model,
      train_loader=train_loader,
      val_loader=val_loader,
      num_epochs=200,
      learning_rate=1e-3
    )

    model_dir = '../saved_models/'
    sh = 'sh'+str(past_sequence_length)

    # Save the trained model
    save_dir=os.path.join(model_dir, 'saved_models_diatrend/5_fold_'+sh+'/')
    os.makedirs(save_dir, exist_ok=True)
    save_model(model, sh+'_'+fold, save_dir)
#   break


## individual evaluation

In [ ]:
past_sequence_length = 12
future_offset = 6
batch_size = 64
max_interval_minutes = 30
sh = 'sh'+str(past_sequence_length)

model_dir = '../saved_models/'
evaluation_dir = '../evaluation/'
data_dir = '../../../ReproGenBG_Dataset/diatrend_processed/'


test_eval = []
for fold in fold_splits.keys():
    print(fold, fold_splits[fold]['test'])
    # Load the saved model
    model = load_model_population(sh+'_'+fold, past_sequence_length, model_class=TransformerEncoder_version2, save_dir=os.path.join(model_dir, 'saved_models_diatrend/5_fold_'+sh+'/'))
    
    for test in fold_splits[fold]['test']:
        test_df = pd.read_csv(os.path.join(data_dir, test))
        uid = test.split('.')[0].split('processed_cgm_data_Subject')[1]
        test_df = test_df.rename(columns={"date": "timestamp"})
        test_df['timestamp'] = test_df['timestamp'].apply(convert_to_datetime)
        test_df = test_df.loc[:, ['timestamp', 'mg/dl']]
        # print(test_df.shape)
#         break
        metrics = evaluate_and_save_metrics_diatrend(
            model=model,
            test_df=test_df,
            save_dir=os.path.join(evaluation_dir, 'evaluation_metrics_diatrend/5_fold_individual_'+sh+'/'),
            past_sequence_length=past_sequence_length,
            future_offset=future_offset,
            batch_size=batch_size,
            max_interval_minutes=max_interval_minutes,
            uid=uid
        )

    test_eval.append([uid, round(metrics['rmse'], 2), round(metrics['mae'], 2), round(metrics['mape'], 2)])

    # print(f"\nResults for population model:")
    print(f"RMSE: {metrics['rmse']:.2f}")
    print(f"MAE: {metrics['mae']:.2f}")
    print(f"MAPE: {metrics['mape']:.2f}%")

  # break

In [ ]:
print(test_eval)

df = pd.DataFrame(test_eval, columns=['test patient', 'RMSE', 'MAE', 'MAPE'])
df.to_csv(os.path.join("../evaluation/", 'evaluation_metrics_diatrend/5_fold_test_eval_'+sh+'.csv'), index=False)

# Train on T1DEXI dataset

## population (archive)

### data preprocess

In [ ]:
data_dir = '../../../ReproGenBG_Dataset/T1DEXI_processed/'

train_df = pd.DataFrame()
test_df = pd.DataFrame()

for file in os.listdir(data_dir):
  if file.endswith('.csv'):
    df = pd.read_csv(os.path.join(data_dir, file))
    # df.drop(columns=['USUBJID'], inplace=True)
    df = df.rename(columns={"LBORRES": "mg/dl", "LBDTC": "timestamp"})
    df['timestamp'] = df['timestamp'].apply(convert_to_datetime)
    df = df.loc[:, ['USUBJID', 'timestamp', 'mg/dl']] # reorder to keep the same format as Diatrend for future training
    num_train = int(len(df) * 0.8)
    cur_train_df = df.iloc[:num_train]
    cur_test_df = df.iloc[num_train:]
    train_df = pd.concat([train_df, cur_train_df])
    test_df = pd.concat([test_df, cur_test_df])
    # break

population_data_dir = '../../../ReproGenBG_Dataset/T1DEXI_population/'
train_df.to_csv(os.path.join(population_data_dir, 'T1DEXI_train.csv'), index=False)
test_df.to_csv(os.path.join(population_data_dir, 'T1DEXI_test.csv'), index=False)

In [ ]:
print(train_df.shape, test_df.shape)
train_df.head(3)

### model train

In [ ]:
population_data_dir = '../../../ReproGenBG_Dataset/T1DEXI_population/'

df = pd.read_csv(population_data_dir + 'T1DEXI_train.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set hyperparameters
past_sequence_length = 12
future_offset = 6
batch_size = 64
max_interval_minutes = 30

# Train model
model = TransformerEncoder_version2(
    past_seq_len=past_sequence_length,
    num_layers=1,
    d_model=512,
    nhead=4,
    input_dim=1,
    dropout=0.2
)
model = model.to(device)

# Create datasets
train_series_list = []
for uid in df['USUBJID'].unique():
    cur_df = df[df['USUBJID'] == uid]
    cur_df.drop(columns=['USUBJID'], inplace=True)
    series_list = split_into_continuous_series(cur_df, past_sequence_length, future_offset, max_interval_minutes)
    train_series_list.extend(series_list)

train_dataset, val_dataset = create_train_val_datasets(
    train_series_list,
    train_ratio=0.8,
    past_seq_len=past_sequence_length,
    future_offset=future_offset
)

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Train model
train_losses, val_losses = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=200,
    learning_rate=1e-3
)

In [ ]:
model_dir = '../saved_models/'

sh = 'sh'+str(past_sequence_length)

# Save the trained model
save_model(model, 'population_'+sh, save_dir=os.path.join(model_dir, 'saved_models_T1DEXI/'))

### individaul evaluation

In [ ]:
past_sequence_length = 12
future_offset = 6
batch_size = 64
max_interval_minutes = 30
sh = 'sh'+str(past_sequence_length)

model_dir = '../saved_models/'
evaluation_dir = '../evaluation/'

# Load the saved model
model = load_model_population('population_'+sh, past_sequence_length, save_dir=os.path.join(model_dir, 'saved_models_T1DEXI'))
test_eval = []

population_data_dir = '../../../ReproGenBG_Dataset/T1DEXI_population/'
test_df = pd.read_csv(population_data_dir + 'T1DEXI_test.csv')
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])

for uid in test_df['USUBJID'].unique():
    cur_df = test_df[test_df['USUBJID'] == uid]
    cur_df.drop(columns=['USUBJID'], inplace=True)
    # Evaluate on test data individually
    metrics = evaluate_and_save_metrics_T1DEXI(
        model=model,
        test_df=cur_df,
        save_dir=os.path.join(evaluation_dir, 'evaluation_metrics_T1DEXI/80_20_individual_'+sh+'/'),
        past_sequence_length=past_sequence_length,
        future_offset=future_offset,
        batch_size=batch_size,
        max_interval_minutes=max_interval_minutes,
        uid=uid
    )

    id = uid
    test_eval.append([id, round(metrics['rmse'], 2), round(metrics['mae'], 2), round(metrics['mape'], 2)])

    # print(f"\nResults for population model:")
    print(f"RMSE: {metrics['rmse']:.2f}")
    print(f"MAE: {metrics['mae']:.2f}")
    print(f"MAPE: {metrics['mape']:.2f}%")

In [ ]:
# print(test_eval)
df = pd.DataFrame(test_eval, columns=['test patient', 'RMSE', 'MAE', 'MAPE'])
df.to_csv(os.path.join(evaluation_dir, 'evaluation_metrics_T1DEXI/80_20_test_eval_'+sh+'.csv'), index=False)

## 5-fold cross validation

### data split

In [ ]:
data_dir = '../../../ReproGenBG_Dataset/T1DEXI_processed/'

fold_splits = create_5fold_splits_T1DEXI(data_dir)
# print(fold_splits)

### model train

In [ ]:
fold_lst = fold_splits.keys()
print(fold_lst)
data_dir = '../../../ReproGenBG_Dataset/T1DEXI_processed/'

for fold in fold_lst:
  train_df = load_train_data_by_fold_T1DEXI(fold, fold_splits, data_dir)
  print(fold, '\ntrain data shape:', train_df.shape)

  # Move model to GPU if available
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  # Set hyperparameters
  past_sequence_length = 24
  future_offset = 6
  batch_size = 64
  max_interval_minutes = 30

  # Train model
  model = TransformerEncoder_version2(
      past_seq_len=past_sequence_length,
      num_layers=1,
      d_model=512,
      nhead=4,
      input_dim=1,
      dropout=0.2
  )
  model = model.to(device)

  # Create datasets
  train_series_list = []
  for uid in train_df['USUBJID'].unique():
      cur_df = train_df[train_df['USUBJID'] == uid]
      cur_df.drop(columns=['USUBJID'], inplace=True)
      series_list = split_into_continuous_series(cur_df, past_sequence_length, future_offset, max_interval_minutes)
      train_series_list.extend(series_list)

  train_dataset, val_dataset = create_train_val_datasets(
      train_series_list,
      train_ratio=0.8,
      past_seq_len=past_sequence_length,
      future_offset=future_offset
  )

  # Create data loaders
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=batch_size)

  # Train model
  train_losses, val_losses = train_model(
      model=model,
      train_loader=train_loader,
      val_loader=val_loader,
      num_epochs=200,
      learning_rate=1e-3
  )

  model_dir = '../saved_models/'
  sh = 'sh'+str(past_sequence_length)

  # Save the trained model
  save_dir=os.path.join(model_dir, 'saved_models_T1DEXI/5_fold_'+sh+'/')
  os.makedirs(save_dir, exist_ok=True)
  save_model(model, sh+'_'+fold, save_dir)

#   break

### individual evaluation

In [ ]:
past_sequence_length = 12
future_offset = 6
batch_size = 64
max_interval_minutes = 30
sh = 'sh'+str(past_sequence_length)
model_dir = '../saved_models/'
evaluation_dir = '../evaluation/'
data_dir = '../../../ReproGenBG_Dataset/T1DEXI_processed/'

test_eval = []
for fold in fold_splits.keys():
  print(fold, fold_splits[fold]['test'])
  # Load the saved model
  model = load_model_population(sh+'_'+fold, past_sequence_length, save_dir=os.path.join(model_dir, 'saved_models_T1DEXI/5_fold_'+sh+'/'))

  for test in fold_splits[fold]['test']:
    uid = test.split('.')[0]
    test_df = pd.read_csv(os.path.join(data_dir, test))
    test_df = test_df.rename(columns={"LBORRES": "mg/dl", "LBDTC": "timestamp"})
    test_df['timestamp'] = test_df['timestamp'].apply(convert_to_datetime)
    test_df = test_df.loc[:, ['timestamp', 'mg/dl']]
    # print(test_df.shape)
    # break
    metrics = evaluate_and_save_metrics_T1DEXI(
        model=model,
        test_df=test_df,
        save_dir=os.path.join(evaluation_dir, 'evaluation_metrics_T1DEXI/5_fold_individual_'+sh+'/'),
        past_sequence_length=past_sequence_length,
        future_offset=future_offset,
        batch_size=batch_size,
        max_interval_minutes=max_interval_minutes,
        uid=uid
    )

    test_eval.append([uid, round(metrics['rmse'], 2), round(metrics['mae'], 2), round(metrics['mape'], 2)])

    # print(f"\nResults for population model:")
    print(f"RMSE: {metrics['rmse']:.2f}")
    print(f"MAE: {metrics['mae']:.2f}")
    print(f"MAPE: {metrics['mape']:.2f}%")

#   break

In [ ]:
# print(test_eval)
evaluation_dir = '../evaluation/'

df = pd.DataFrame(test_eval, columns=['test patient', 'RMSE', 'MAE', 'MAPE'])
df.to_csv(os.path.join(evaluation_dir, 'evaluation_metrics_T1DEXI/5_fold_test_eval_'+sh+'.csv'), index=False)

# Train on Ohio dataset

## Population data

In [ ]:
data_dir = '../../../ReproGenBG_Dataset/'
folder_path_train_2018 = os.path.join(data_dir, "./OhioT1DM 2020/2018/train")
folder_path_train_2020 = os.path.join(data_dir,"./OhioT1DM 2020/2020/train")
train_files_2018 = [f for f in os.listdir(folder_path_train_2018) if f.endswith('.xml')]
train_files_2020 = [f for f in os.listdir(folder_path_train_2020) if f.endswith('.xml')]

folder_path_test_2018 = os.path.join(data_dir,"./OhioT1DM 2020/2018/test")
folder_path_test_2020 = os.path.join(data_dir,"./OhioT1DM 2020/2020/test")
test_files_2018 = [f for f in os.listdir(folder_path_test_2018) if f.endswith('.xml')]
test_files_2020 = [f for f in os.listdir(folder_path_test_2020) if f.endswith('.xml')]

population_splits = create_population_splits(
    folder_path_train_2018,
    folder_path_train_2020,
    train_files_2018,
    train_files_2020,
    folder_path_test_2018,
    folder_path_test_2020,
    test_files_2018,
    test_files_2020
)

print(population_splits)

In [ ]:
# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set hyperparameters
past_sequence_length = 24
future_offset = 6
batch_size = 64
max_interval_minutes = 30

# Train model
model = TransformerEncoder_version2(
    past_seq_len=past_sequence_length,
    num_layers=1,
    d_model=512,
    nhead=4,
    input_dim=1,
    dropout=0.2
)
model = model.to(device)

# Load and process training data
train_dfs = []
for train_file in population_splits['train']:
    df = load_ohio_series_train(train_file, "glucose_level", "value")
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    train_dfs.append(df)

# Create datasets
train_series_list = []
for df in train_dfs:
    series_list = split_into_continuous_series(df, past_sequence_length, future_offset, max_interval_minutes)
    train_series_list.extend(series_list)

train_dataset, val_dataset = create_train_val_datasets(
    train_series_list,
    train_ratio=0.8,
    past_seq_len=past_sequence_length,
    future_offset=future_offset
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Train model
train_losses, val_losses = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=200,
    learning_rate=1e-3
)

model_dir = '../saved_models/'
sh = 'sh'+str(past_sequence_length)

# Save the trained model
save_model(model, 'population_'+sh, save_dir=os.path.join(model_dir, 'saved_models_original_ohio/'))

In [ ]:
past_sequence_length = 24
future_offset = 6
batch_size = 64
max_interval_minutes = 30
evaluation_dir = '../evaluation/'

sh = 'sh'+str(past_sequence_length)
model = load_model_population('population_'+sh, past_sequence_length, save_dir=os.path.join(model_dir, 'saved_models_original_ohio'))

# Evaluate on test data
metrics = evaluate_and_save_metrics_population(
    model=model,
    test_file_paths=population_splits['test'],
    save_dir=os.path.join(evaluation_dir, 'evaluation_metrics_original_ohio'),
    past_sequence_length=past_sequence_length,
    future_offset=future_offset,
    batch_size=batch_size,
    max_interval_minutes=max_interval_minutes
)

# evaluation on whole test set
print(f"RMSE: {metrics['rmse']:.2f}")
print(f"MAE: {metrics['mae']:.2f}")
print(f"MAPE: {metrics['mape']:.2f}%")

In [ ]:
# Load the saved model
model = load_model_population('population_'+sh, past_sequence_length, save_dir=os.path.join(model_dir, 'saved_models_original_ohio'))
test_eval = []

for test in population_splits['test']:
    print(test)
    # Evaluate on test data individually
    metrics = evaluate_and_save_metrics(
        model=model,
        test_file_path=test,
        save_dir=os.path.join(evaluation_dir, 'evaluation_metrics_original_ohio/individual_'+sh+'/'),
        past_sequence_length=past_sequence_length,
        future_offset=future_offset,
        batch_size=batch_size,
        max_interval_minutes=max_interval_minutes
    )

    id = test.split('/')[-1].split('-')[0]
    test_eval.append([id, round(metrics['rmse'], 2), round(metrics['mae'], 2), round(metrics['mape'], 2)])

    # print(f"\nResults for population model:")
    print(f"RMSE: {metrics['rmse']:.2f}")
    print(f"MAE: {metrics['mae']:.2f}")
    print(f"MAPE: {metrics['mape']:.2f}%")

### save individual results to .csv file

In [ ]:
print(test_eval)
evaluation_dir = '../evaluation/'

df = pd.DataFrame(test_eval, columns=['test patient', 'RMSE', 'MAE', 'MAPE'])
df.to_csv(os.path.join(evaluation_dir, 'evaluation_metrics_original_ohio/individual_test_eval_'+sh+'.csv'), index=False)

# Ohio 60 mins model train on diatrend and T1DEXI

In [ ]:
past_sequence_length = 12
future_offset = 6
batch_size = 64
max_interval_minutes = 30

model_dir = '../saved_models/'
sh = 'sh'+str(past_sequence_length)
model = load_model_population('population_'+sh, past_sequence_length, save_dir=os.path.join(model_dir, 'saved_models_original_ohio'))

## Diatrend

In [ ]:
past_sequence_length = 12
future_offset = 6
batch_size = 64
max_interval_minutes = 30

model_dir = '../saved_models/'

sh = 'sh'+str(past_sequence_length)
model = load_model_population('population_'+sh, past_sequence_length, save_dir=os.path.join(model_dir, 'saved_models_original_ohio'))

In [ ]:
data_dir = '../../../ReproGenBG_Dataset/diatrend_processed/'
test_eval = []

for test in os.listdir(data_dir):
  test_df = pd.read_csv(os.path.join(data_dir, test))
  uid = test.split('.')[0].split('processed_cgm_data_Subject')[1]
  test_df = test_df.rename(columns={"date": "timestamp"})
  test_df['timestamp'] = test_df['timestamp'].apply(convert_to_datetime)
  test_df = test_df.loc[:, ['timestamp', 'mg/dl']]
  # print(test_df.shape)
  # break
  metrics = evaluate_and_save_metrics_diatrend(
      model=model,
      test_df=test_df,
      save_dir=os.path.join(evaluation_dir, 'evaluation_metrics_original_ohio/individual_t1dexi_diatrend/diatrend_'+sh+'/'),
      past_sequence_length=past_sequence_length,
      future_offset=future_offset,
      batch_size=batch_size,
      max_interval_minutes=max_interval_minutes,
      uid=uid
  )

  test_eval.append([uid, round(metrics['rmse'], 2), round(metrics['mae'], 2), round(metrics['mape'], 2)])

  # print(f"\nResults for population model:")
  print(f"RMSE: {metrics['rmse']:.2f}")
  print(f"MAE: {metrics['mae']:.2f}")
  print(f"MAPE: {metrics['mape']:.2f}%")


In [ ]:
print(test_eval)
df = pd.DataFrame(test_eval, columns=['test patient', 'RMSE', 'MAE', 'MAPE'])
df.to_csv(os.path.join(evaluation_dir, 'evaluation_metrics_original_ohio/individual_t1dexi_diatrend/diatrend_eval_'+sh+'.csv'), index=False)

## T1DEXI

In [ ]:
test_eval = []
data_dir = '../../../ReproGenBG_Dataset/T1DEXI_processed/'

for test in os.listdir(data_dir):

  uid = test.split('.')[0]
  test_df = pd.read_csv(os.path.join(data_dir, test))
  test_df = test_df.rename(columns={"LBORRES": "mg/dl", "LBDTC": "timestamp"})
  test_df['timestamp'] = test_df['timestamp'].apply(convert_to_datetime)
  test_df = test_df.loc[:, ['timestamp', 'mg/dl']]
  # print(test_df.shape)
  # break
  metrics = evaluate_and_save_metrics_T1DEXI(
      model=model,
      test_df=test_df,
      save_dir=os.path.join(evaluation_dir, 'evaluation_metrics_original_ohio/individual_t1dexi_diatrend/t1dexi_'+sh+'/'),
      past_sequence_length=past_sequence_length,
      future_offset=future_offset,
      batch_size=batch_size,
      max_interval_minutes=max_interval_minutes,
      uid=uid
  )

  test_eval.append([uid, round(metrics['rmse'], 2), round(metrics['mae'], 2), round(metrics['mape'], 2)])

  print(f"RMSE: {metrics['rmse']:.2f}")
  print(f"MAE: {metrics['mae']:.2f}")
  print(f"MAPE: {metrics['mape']:.2f}%")

In [ ]:
# print(test_eval)
df = pd.DataFrame(test_eval, columns=['test patient', 'RMSE', 'MAE', 'MAPE'])
df.to_csv(os.path.join(evaluation_dir, 'evaluation_metrics_original_ohio/individual_t1dexi_diatrend/t1dexi_eval_'+sh+'.csv'), index=False)